In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# load&overview Data

In [ ]:
df_train = pd.read_csv('/kaggle/input/tabular-playground-series-aug-2021/train.csv')
df_test = pd.read_csv('/kaggle/input/tabular-playground-series-aug-2021/test.csv')
df_simple_sub = pd.read_csv('/kaggle/input/tabular-playground-series-aug-2021/sample_submission.csv')


In [ ]:
df_train

In [ ]:
df_test.info()

In [ ]:
df_train.isnull().sum().sum()

In [ ]:
df_test.isnull().sum().sum()

In [ ]:
fea = df_test.columns.to_list()
fea.remove('id')

In [ ]:
from tensorflow.keras.layers.experimental import preprocessing
normalizer = preprocessing.Normalization(axis=-1)
normalizer.adapt(np.array(df_train[fea]))

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
def build_and_compile_model(norm):
  model = keras.Sequential([
      norm,

      # layers.BatchNormalization(),
      layers.Dense(128, activation='relu'),
      # layers.Dropout(0.2),
      
      # layers.BatchNormalization(),
      layers.Dense(64, activation='relu'),
      # layers.Dropout(0.2),

      layers.Dense(32, activation='relu'),
      # layers.Dropout(0.2),

      layers.Dense(1)
  ])

  model.compile(loss='mse',
                optimizer=tf.keras.optimizers.Adam(0.001))
  return model

In [ ]:
dnn_model = build_and_compile_model(normalizer)
dnn_model.summary()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(
    min_delta=0.001,
    patience=5,
    restore_best_weights=True,
)

In [ ]:
%%time
history = dnn_model.fit(
    x=df_train[fea], 
    y=df_train['loss'],
    validation_split=0.2,
    batch_size=20,
    # verbose=0, 
    epochs=10,
    callbacks=[es])

In [ ]:
# 没有考虑精度前的结果
y_test_pred = dnn_model.predict(df_test[fea])
df_simple_sub['loss'] = y_test_pred
df_simple_sub.to_csv('simple_dnn_submission.csv', index=False)